## Análisis Exploratorio de Datos (EDA)

In [2]:
# Importar librerías necesarias
import pandas as pd
import sqlite3 as sql

### Cargar datos

In [13]:
import pandas as pd
import sqlite3 as sql

ruta_archivo = "../data/raw/adidas_us_sales_dataset.xlsx"
df = pd.read_excel(ruta_archivo, index_col=None)

# Eliminar columna de índice anterior si existe
if 'Unnamed: 0' in df.columns:
    df.drop(columns=['Unnamed: 0'], inplace=True)

# Guardar en SQLite
conn = sql.connect("../data/raw/adidas_sales.db")
df.to_sql("ventas_adidas", conn, if_exists="replace", index=False)

print("Base de datos creada con éxito")

cursor = conn.cursor()


Base de datos creada con éxito


In [ ]:
#conn.close()

In [15]:
consulta = """
        SELECT *
        FROM ventas_adidas
        LIMIT 5;
        """
consulta = pd.read_sql_query(consulta, conn)
consulta

,Retailer,Retailer ID,Invoice Date,Region,State,City,Product,Price per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method
0,Foot Locker,1185732,1969-12-31 19:00:43.831000,Northeast,New York,New York,Men's Street Footwear,50.0,1200,600000.0,300000.0,0.50,In-store
1,Foot Locker,1185732,1969-12-31 19:00:43.832000,Northeast,New York,New York,Men's Athletic Footwear,50.0,1000,500000.0,150000.0,0.30,In-store
2,Foot Locker,1185732,1969-12-31 19:00:43.833000,Northeast,New York,New York,Women's Street Footwear,40.0,1000,400000.0,140000.0,0.35,In-store
3,Foot Locker,1185732,1969-12-31 19:00:43.834000,Northeast,New York,New York,Women's Athletic Footwear,45.0,850,382500.0,133875.0,0.35,In-store
4,Foot Locker,1185732,1969-12-31 19:00:43.835000,Northeast,New York,New York,Men's Apparel,60.0,900,540000.0,162000.0,0.30,In-store
